In [ ]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)

In [ ]:
y = y['income'].map({'<=50K':0,'<=50K.':0,'>50K':1,'>50K.':1})
X = X[['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']]

y = y.values
X = X.values.astype('float64')

X -= np.mean(X,axis=0)

X = torch.tensor(X).float()
y = torch.tensor(y).long()

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# 2 MLP Model (with one hidden layer)
mlp_model = torch.nn.Sequential(
    torch.nn.Linear(6, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 2),
)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
lr = 1e-2
opt = torch.optim.SGD(mlp_model.parameters(), weight_decay=1e-4, lr=lr)

num_epochs = 100

for epoch in range(num_epochs):
    for batch_idx, (X, y) in enumerate(train_loader):
            opt.zero_grad()
            z = mlp_model(X) # compute z values
            loss = loss_fn(z,y) # compute loss

            loss.backward() # compute gradients

            opt.step() # apply gradients

    print(f'epoch {epoch}: loss is {loss.item()}')

